<a href="https://colab.research.google.com/github/LeeGitaek/2020_AI_Class/blob/master/MNIST_Dnn_Xavier_dropout_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

Uninstalling kaggle-1.5.6:
  Would remove:
    /usr/local/bin/kaggle
    /usr/local/lib/python3.6/dist-packages/kaggle-1.5.6.dist-info/*
    /usr/local/lib/python3.6/dist-packages/kaggle/*
Proceed (y/n)? y
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.5MB 9.0MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 3.9 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=c555ad03fa34dd6703106b103c49bc533d81beb264a3f1a6ea8c5f8f660e922a
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle


In [9]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!ls -lha kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 68 Jun  5 11:32 kaggle.json


In [10]:
!kaggle competitions download -c 2020aimnist

 51% 9.00M/17.8M [00:00<00:00, 21.3MB/s]
100% 17.8M/17.8M [00:00<00:00, 39.6MB/s]


In [11]:
!unzip 2020aimnist.zip

Archive:  2020aimnist.zip
  inflating: mnist_test.csv          
  inflating: mnist_train.csv         
  inflating: submission.csv          


In [0]:
import pandas as pd
import numpy as np

import torch
import torchvision.datasets as data 
import torchvision.transforms as transforms
import random 

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
random.seed(777)
#난수생성 규칙 설정 seed
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [0]:
#학습 파라미터 설정
# tpu = 구글에서 만든 gpu

learning_rate = 1e-4
training_epochs = 100
batch_size = 100
drop_prob = 0.3
#모의 문제 해결 수 

In [0]:
train_data=pd.read_csv('mnist_train.csv',header=None,skiprows=1, usecols=range(1,786))
test_data=pd.read_csv('mnist_test.csv',header=None,skiprows=1, usecols=range(1,785))

In [0]:
x_train_data=train_data.loc[:,0:784]
y_train_data=train_data.loc[:,785]

x_train_data=np.array(x_train_data)
y_train_data=np.array(y_train_data)

x_train_data=torch.FloatTensor(x_train_data)
y_train_data=torch.LongTensor(y_train_data)

In [0]:
train_dataset = torch.utils.data.TensorDataset(x_train_data, y_train_data)


In [0]:
data_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)


#shuffle = 데이터 섞어줌
## root 값은 어디에 저장할 것인가 

In [0]:
linear1 = torch.nn.Linear(784,512,bias=True)
linear2 = torch.nn.Linear(512,512,bias=True)
linear3 = torch.nn.Linear(512,512,bias=True)
linear4 = torch.nn.Linear(512,512,bias=True)
linear5 = torch.nn.Linear(512,10,bias=True)
relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p=drop_prob)
#레이어를 5개 쌓는다. 
# ( nn + relu ) *5

In [280]:
torch.nn.init.xavier_uniform(linear1.weight)
torch.nn.init.xavier_uniform(linear2.weight)
torch.nn.init.xavier_uniform(linear3.weight)
torch.nn.init.xavier_uniform(linear4.weight)
torch.nn.init.xavier_uniform(linear5.weight)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  """


Parameter containing:
tensor([[ 0.0972,  0.1054,  0.0534,  ..., -0.0436, -0.1015,  0.0360],
        [-0.0800, -0.0816,  0.0367,  ..., -0.0784, -0.1051,  0.0818],
        [-0.0354,  0.0964, -0.0991,  ..., -0.0918, -0.0482, -0.0820],
        ...,
        [ 0.0556,  0.0444,  0.0962,  ...,  0.0452,  0.0126, -0.0811],
        [ 0.0297,  0.0645, -0.0285,  ...,  0.0215, -0.0248,  0.0638],
        [ 0.0596, -0.0491,  0.0391,  ..., -0.0817,  0.0132, -0.0201]],
       requires_grad=True)

In [0]:
model = torch.nn.Sequential(linear1,relu,dropout,
                            linear2,relu,dropout,
                            linear3,relu,dropout,
                            linear4,relu,dropout,
                            linear5).to(device)
#맨 마지막 레이어에서는 relu는 빼준다.

In [0]:
#손실함수와 최적화 함수 
loss = torch.nn.CrossEntropyLoss().to(device) # 소프트맥스 내부적으로 계산 
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate) 

In [283]:

total_batch = len(data_loader)
model.train() # drop_out= true
for epoch in range(training_epochs):
    avg_cost = 0

    for X,Y in data_loader:
      # (100,1,28,28) 크기의 텐서를 (100,784) 크기의 텐서로 변형 
       X = X.view(-1,28*28).to(device) #reshape과 유사
       # one - hot 인코딩 되어있지 않음 
       Y = Y.to(device)

       # 그레디언트 초기화
       optimizer.zero_grad()
       hypothesis = model(X)

       cost = loss(hypothesis,Y) 
       cost.backward()
       optimizer.step()

       avg_cost += cost/total_batch

    print('Epoch :','%04d' % (epoch+1),'cost=','{:.9f}'.format(avg_cost))
print('Learning finished..!')

Epoch : 0001 cost= 9.880663872
Epoch : 0002 cost= 1.839114547
Epoch : 0003 cost= 1.044443727
Epoch : 0004 cost= 0.732924700
Epoch : 0005 cost= 0.575284123
Epoch : 0006 cost= 0.464927405
Epoch : 0007 cost= 0.395694345
Epoch : 0008 cost= 0.336847603
Epoch : 0009 cost= 0.296014965
Epoch : 0010 cost= 0.262776673
Epoch : 0011 cost= 0.233232498
Epoch : 0012 cost= 0.210859716
Epoch : 0013 cost= 0.188492045
Epoch : 0014 cost= 0.163694307
Epoch : 0015 cost= 0.153030246
Epoch : 0016 cost= 0.140162721
Epoch : 0017 cost= 0.129699826
Epoch : 0018 cost= 0.118920758
Epoch : 0019 cost= 0.109271824
Epoch : 0020 cost= 0.101912305
Epoch : 0021 cost= 0.093531877
Epoch : 0022 cost= 0.088031881
Epoch : 0023 cost= 0.083170332
Epoch : 0024 cost= 0.075854741
Epoch : 0025 cost= 0.071299292
Epoch : 0026 cost= 0.064971887
Epoch : 0027 cost= 0.064005941
Epoch : 0028 cost= 0.058207151
Epoch : 0029 cost= 0.059043184
Epoch : 0030 cost= 0.052417383
Epoch : 0031 cost= 0.049949266
Epoch : 0032 cost= 0.048953287
Epoch : 

In [0]:

with torch.no_grad():
  model.eval()
  x_test_data=test_data.loc[:,:]
  x_test_data=np.array(x_test_data)
  x_test_data=torch.from_numpy(x_test_data).float().to(device)

  prediction = model(x_test_data)
  correct_prediction = torch.argmax(prediction, 1)


In [0]:
correct_prediction = correct_prediction.cpu().numpy().reshape(-1,1)

In [286]:
submit=pd.read_csv('submission.csv')
submit

,id,Label
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
9995,9995,1
9996,9996,1
9997,9997,1
9998,9998,1


In [287]:
for i in range(len(correct_prediction)):
  submit['Label'][i]=correct_prediction[i].item()

submit

,id,Label
0,0,7
1,1,2
2,2,1
3,3,0
4,4,4
...,...,...
9995,9995,2
9996,9996,3
9997,9997,4
9998,9998,5


In [288]:
submit.to_csv('submission_result.csv',index=False,header=True)

!kaggle competitions submit -c 2020aimnist -f submission_result.csv -m "14010974 이기택"

100% 67.3k/67.3k [00:03<00:00, 22.2kB/s]
Successfully submitted to 2020.AI.MNIST